In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [15]:
# Load email data from emails.csv
data = pd.read_csv('sample_data/emails.csv')
X_text = data['text']
y = data['spam'].values.astype(np.float32)

# Convert text data to TF-IDF features
vectorizer = TfidfVectorizer(max_features=100)
X = vectorizer.fit_transform(X_text).toarray().astype(np.float32)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameters
input_size = 100
hidden_size = 50
output_size = 1
epochs = 10
batch_size = 32

# PyTorch Model
class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# TensorFlow Model
tf_model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_size, activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(output_size, activation='sigmoid')
])
tf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# PyTorch Training and Evaluation
def train_pytorch_model():
    model = PyTorchModel()
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    start_time = time.time()
    for epoch in range(epochs):
        for i in range(0, len(X_train), batch_size):
            batch_X = torch.tensor(X_train[i:i+batch_size])
            batch_y = torch.tensor(y_train[i:i+batch_size]).float()

            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.view(-1, 1))
            loss.backward()
            optimizer.step()
    end_time = time.time()
    print(f'PyTorch Training Time: {end_time - start_time:.2f} seconds')

    # Evaluate on test set
    model.eval()
    with torch.no_grad():
        test_X = torch.tensor(X_test)
        test_outputs = model(test_X).numpy()
        test_predictions = (test_outputs > 0.5).astype(np.float32)
        accuracy = accuracy_score(y_test, test_predictions)
        print(f'PyTorch Test Accuracy: {accuracy:.2f}')

# TensorFlow Training and Evaluation
def train_tensorflow_model():
    start_time = time.time()
    tf_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    end_time = time.time()
    print(f'TensorFlow Training Time: {end_time - start_time:.2f} seconds')

    # Evaluate on test set
    test_predictions = (tf_model.predict(X_test) > 0.5).astype(np.float32)
    accuracy = accuracy_score(y_test, test_predictions)
    print(f'TensorFlow Test Accuracy: {accuracy:.2f}')

# Compare Training Performance
train_pytorch_model()
train_tensorflow_model()

# Compare Inference Performance
sample_input = np.random.randn(1, input_size).astype(np.float32)

# PyTorch Inference
def pytorch_inference():
    model = PyTorchModel()
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(sample_input)
        start_time = time.time()
        for _ in range(1000):
            output = model(input_tensor)
        end_time = time.time()
    print(f'PyTorch Inference Time (1000 runs): {end_time - start_time:.2f} seconds')

# TensorFlow Inference
def tensorflow_inference():
    start_time = time.time()
    for _ in range(1000):
        output = tf_model(sample_input)
    end_time = time.time()
    print(f'TensorFlow Inference Time (1000 runs): {end_time - start_time:.2f} seconds')

# Compare Inference Performance
pytorch_inference()
tensorflow_inference()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


PyTorch Training Time: 2.25 seconds
PyTorch Test Accuracy: 0.96
TensorFlow Training Time: 3.99 seconds
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
TensorFlow Test Accuracy: 0.96
PyTorch Inference Time (1000 runs): 0.05 seconds
TensorFlow Inference Time (1000 runs): 2.41 seconds
